In [1]:
%matplotlib inline

In [2]:
import utils
from utils import *

Using Theano backend.


In [72]:
from __future__ import division, print_function
import os, json
from glob import glob

from matplotlib import pyplot as plt

import scipy
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import numpy as np
from numpy.random import random, permutation

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

np.set_printoptions(precision=4, linewidth=100)

In [73]:
def onehot(x):
    return np.array( OneHotEncoder().fit_transform(x.reshape(-1, 1)).todense() )

In [74]:
path='data/dogscats/sample/'
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [75]:
batch_size = 4

In [76]:
from vgg16 import Vgg16

In [77]:
vgg = Vgg16()

In [78]:
model = vgg.model

In [79]:
validation_batches = get_batches(path+'valid', shuffle=False, batch_size=1)

Found 20 images belonging to 2 classes.


In [80]:
training_batches = get_batches(path+'train', shuffle=True, batch_size=1)

Found 42 images belonging to 2 classes.


In [81]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_14[0][0]           
___________________________________________________________________________________________

In [82]:
model.pop()

In [83]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_14[0][0]           
___________________________________________________________________________________________

In [95]:
for layer in model.layers: layer.trainable = False

In [85]:
model.add(Dense(2, activation='softmax'))

In [86]:
opt = RMSprop(lr=0.1)

In [87]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [88]:
model.fit_generator(training_batches, samples_per_epoch=5, nb_epoch=1, validation_data=validation_batches, nb_val_samples=5)

Epoch 1/1
5/5 [==============================] - 24s - loss: 6.4483 - acc: 0.6000 - val_loss: 1.1921e-07 - val_acc: 1.0000


In [89]:
validation_data = get_data(path+'valid')
validation_classes = validation_batches.classes
validation_labels = onehot(validation_classes)

model.evaluate(validation_data, validation_labels)

Found 20 images belonging to 2 classes.
20/20 [==============================] - 10s


[8.0590476989746094, 0.5]

In [90]:
validation_data.shape

(20, 3, 224, 224)

In [91]:
model.predict_classes(validation_data)

20/20 [==============================] - 10s


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [94]:
validation_classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [66]:
validation_batches.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)